<a href="https://colab.research.google.com/github/ferjjp/tp-iaa-clasificacion-de-manos/blob/main/TP_IAA_Reconocimiento_de_gestos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preparar ambiente

In [ ]:
#@title Instalar Auto-Keras 
#!pip install git+https://github.com/keras-team/keras-tuner.git
!pip install keras-tuner --upgrade
!pip install autokeras

In [ ]:
#@title Librerías a usar
%tensorflow_version 2.x
import autokeras as ak

from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os

from PIL import Image

from  sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print("Librerías cargadas")

print("Setupeando TPU")
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)

In [ ]:
## selección de los parámetros 

#@markdown ### Parámetros de imágenes:
imagen_largo = 50 #@param {type:"integer"}
imagen_ancho = 50 #@param {type:"integer"}
imagen_color = True #@param {type:"boolean"}
imagen_usar_generadas_data_augmentation = True #@param {type:"boolean"}

## aplicación de los parámetros elegidos

# tamaño de las imágenes
IMAGE_SHAPE = (imagen_largo, imagen_ancho, (3 if imagen_color else 1))

# indica si se usan las imágenes generadas por data augmentation
usarDA = imagen_usar_generadas_data_augmentation

# define tamaño de datos de entrada 
num_inputs = IMAGE_SHAPE[0] * IMAGE_SHAPE[1] * IMAGE_SHAPE[2]

print ("Tamaño Imagen: ", IMAGE_SHAPE)

In [ ]:
#@title Donde estan las imagenes?

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# directorio local en Google Drive
path = 'gdrive/My Drive/IA_TP/Fotos/fase2_fer/' #@param {type:"string"}
path_entrenamiento = '/Entrenamiento'  #@param {type:"string"}
path_prueba = '/Pruebas'  #@param {type:"string"}

imagPath_train = path + path_entrenamiento
imagPath_test = path + path_prueba

In [ ]:
#@title: Definicion del dataset
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
   imagPath_train,
   validation_split=0.2,
  #  label_mode='categorical',
   subset="training",
   seed=123,
   image_size=(imagen_largo, imagen_ancho),
   batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
   imagPath_train,
   validation_split=0.2,
  #  label_mode='categorical',
   subset="validation",
   seed=123,
   image_size=(imagen_largo, imagen_ancho),
   batch_size=batch_size
)


class_names = train_ds.class_names

AUTOTUNE = tf.data.AUTOTUNE
def preprocess(ds):
  return ds.cache().prefetch(buffer_size=AUTOTUNE)
train_ds = preprocess(train_ds) #prefetch ciertas imagenes
val_ds = preprocess(val_ds)

In [ ]:
#@title Definicion de modelo

tipo_modelo_usar = "ImageClassifier" #@param ["ImageClassifier", "Personalizado"]

cantidad_intentos_encontrar_modelo = 5 #@param {type:"integer"}
if cantidad_intentos_encontrar_modelo < 1:
  cantidad_intentos_encontrar_modelo = 1

max_cant_params_modelo =  50000000#@param {type:"integer"}
if max_cant_params_modelo <= 0:
  max_cant_params_modelo = None

max_epocas_entrenamiento =  300#@param {type:"integer"}
if max_epocas_entrenamiento <= 0:
  max_epocas_entrenamiento = None

with strategy.scope():
  if tipo_modelo_usar == "ImageClassifier":
      # Initialize the image classifier.
      AKmodel = ak.ImageClassifier( num_classes=len(class_names), 
                                            overwrite=True, 
                                            seed=1,
                                            objective='val_accuracy')#,
                                            # max_model_size=max_cant_params_modelo,
                                            # max_trials=cantidad_intentos_encontrar_modelo)
  else:
      # capa de entrada
      input_node = ak.ImageInput()
      # capas intermedias
      output_node = ak.ImageBlock(
          # Only search ConvNet architectures.
          block_type="vanilla",
          # Normalize the dataset.
          normalize=True,
          # Allow do data augmentation.
          augment=True,
      )(input_node)
      # capa de salida
      # 
      output_node = ak.ClassificationHead(num_classes=len(class_names))(output_node)
      # Initialize AutoModel personalizado
      AKmodel = ak.AutoModel(
          inputs=input_node, outputs=output_node,          
          overwrite=True, 
          seed=1,
          objective='val_accuracy',
          max_model_size=max_cant_params_modelo,
          max_trials=cantidad_intentos_encontrar_modelo)

print("Modelo preparado")

In [ ]:
#@title Entrenar con AutoKeras

# el history sólo devuelve el del último trial
history = AKmodel.fit(train_ds,
            epochs=max_epocas_entrenamiento,
            verbose=1,
            validation_data=val_ds)

print("\n>Entrenamiento Finalizado.")

In [ ]:
#@title Armar resumen del tuner
print(AKmodel.tuner.results_summary())

In [ ]:
#@title Evaluamos el modelo contra el dataset

resEval = AKmodel.evaluate(test_ds)
print("\n>Evaluación del Mejor Modelo con datos de Prueba: ")
print("    - Error: ", resEval[0])
print("    - Exactitud: ", resEval[1]*100)
print("\n")

In [ ]:
#@title Levantamos el dataset de test
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
   imagPath_test,
   seed=123,
   image_size=(imagen_largo, imagen_ancho),
   batch_size=batch_size
)

test_class_names = test_ds.class_names

test_ds = preprocess(test_ds)

In [ ]:
#@title Exportar Modelo y Re-Entrenar (opcional)

reentrenar_modelo = False #@param {type:"boolean"}

cant_epocas_reentrenamiento =  500#@param {type:"integer"}
if cant_epocas_reentrenamiento <= 0:
  cant_epocas_reentrenamiento = None


# exporta el modelo y lo muestra
print("\n>> Mejor modelo generado: ")
model = AKmodel.export_model()
model.summary()
print()
print("==================================\n")

tf.keras.utils.plot_model(
    model, to_file='RNA.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)

if reentrenar_modelo: 
  # realiza el re-entrenamiento del modelo
  print("\n\n>Comienza el Re-Entrenamiento:")
  history = model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=cant_epocas_reentrenamiento
                        )
  print("\n>Re-Entrenamiento Finalizado.")

#@title Mostrar Gráficos del Entrenamiento
plt.figure(figsize=(15,8)) 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Gráfico del Error del Entrenamiento')
plt.ylabel('')
plt.xlabel('epoch')
plt.legend(['entrenamiento', 'validación'], loc='upper left')
plt.show()

plt.figure(figsize=(15,8)) 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Gráfico de la Exactitud del Entrenamiento')
plt.ylabel('')
plt.xlabel('epoch')
plt.legend(['entrenamiento', 'validación'], loc='upper left')
plt.show()

In [ ]:
#@title Evaluar modelo contra test dataset

test_images_normal = []
test_images = []
test_images_classes = []

for image, label in test_ds:
  # for displaying purposes
  test_images_normal.append(image)
  test_images_classes.append(label.numpy())

predictions = model.predict(test_ds)

test_predict = np.argmax(predictions,axis=1)
flattened_real =[item for sublist in test_images_classes for item in sublist]
test_real = np.array(flattened_real)

test_acc = sum(1 if x == y else 0 for x, y in zip(test_predict.tolist(),test_real)) / len(test_real)                                       
print(f'Test set accuracy: {test_acc:.0%}\n')

#----
import seaborn as sns
from sklearn.metrics import confusion_matrix

confusion_mtx = tf.math.confusion_matrix(test_real, test_predict)

print("Reporte de clasificacion\n")
print(classification_report(flattened_real,test_predict))

test_real_labels = list(map(lambda x: class_names[x],test_real))
test_predict_labels = list(map(lambda x: class_names[x],test_predict.tolist()))

cm = confusion_matrix(test_real_labels,test_predict_labels,labels=class_names)
cmtx = pd.DataFrame(
        cm, 
        index=['r:{:}'.format(x) for x in class_names], 
        columns=['p:{:}'.format(x) for x in class_names]
      )
print(cmtx)
print("\n")


print("Heatmap de confusion o matriz\n")
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, xticklabels=class_names, yticklabels=class_names, 
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Class')
plt.show()

print("Predicciones de ejemplo")
plt.figure(figsize=(50, 50))
position = 0
for batch in test_images_normal:
  for image in batch:
    prediction = tf.nn.softmax(predictions[position])
    label = flattened_real[position]
    position = position + 1
    ax = plt.subplot(len(batch),1, position)
    plt.imshow(image.numpy().astype("uint8"))
    plt.title("Predicted {} with {:.2f}% confidence, and it was {}"
               .format(class_names[np.argmax(prediction)],100 * np.max(prediction),class_names[label]))
    plt.axis("off")